***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-02cc  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import numpy as np
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 10 11:37 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'natnat' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh
postings_gcp/0_000.bin
postings_gcp/0_001.bin
postings_gcp/0_002.bin
postings_gcp/0_003.bin
postings_gcp/0_004.bin
postings_gcp/0_005.bin
postings_gcp/0_006.bin
postings_gcp/0_007.bin
postings_gcp/0_008.bin
postings_gcp/0_009.bin
postings_gcp/0_010.bin
postings_gcp/0_011.bin
postings_gcp/0_012.bin
postings_gcp/0_013.bin
postings_gcp/0_014.bin
postings_gcp/0_015.bin
postings_gcp/0_016.bin
postings_gcp/0_017.bin
postings_gcp/0_018.bin
postings_gcp/0_019.bin
postings_gcp/0_020.bin
postings_gcp/0_021.bin
postings_gcp/0_022.bin
postings_gcp/0_023.bin
postings_gcp/0_024.bin
postings_gcp/0_025.bin
postings_gcp/0_026.bin
postings_gcp/0_027.bin
postings_gcp/0_028.bin
postings_gcp/0_posting_locs.pickle
postings_gcp/100_000.bin
postings_gcp/100_001.bin
postings_gcp/100_002.bin
postings_gcp/100_003.bin
postings_gcp/100_004.bin
postings_gcp/100_005.bin
postings_gcp/100_006.bin
postings_gcp/100_007.bin
postings_gcp/100_008.bin
postings_gcp/100_009.bin
postings_gcp/100_010.bin
postings

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
#chnaged
full_path = "gs://wikidata_preprocessed/*"
corpus_df = spark.read.parquet(full_path)

# take the 'text'/'title/'anchor_text' and 'id' and create an RDD from it
doc_text_pairs = corpus_df.select("text", "id").rdd
doc_title_pairs = corpus_df.select("title", "id").rdd
doc_anchor_pairs_bad_format = corpus_df.select("anchor_text", "id").rdd
doc_anchor_pairs = doc_anchor_pairs_bad_format.map(lambda x: Row(anchor_text= " ".join([str(r.text) for r in x.anchor_text]), id = x.id))

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
corpus_df.count()

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [9]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [10]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [11]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [12]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token, offset):
  return int(_hash(token),16) % NUM_BUCKETS + offset


def tokenization(text):
    no_stop_words = []
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    for token in tokens:
        if token in all_stopwords:
          continue
        no_stop_words.append(token)
    return no_stop_words


# PLACE YOUR CODE HERE
def doc_length(text,id):
  no_stop_words = []
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  count = 0
  for token in tokens:
    if token in all_stopwords:
      continue
    count += 1
  return (id,count)


def word_count(id, tokens):
  # YOUR CODE HERE
  counter = Counter(tokens)
  return list(map(lambda x:(x,(id,counter[x])),counter))

def reduce_word_counts(unsorted_pl):
  # YOUR CODE HERE
  return sorted(unsorted_pl,key = lambda x: x[0])

def calculate_df(postings):
  # YOUR CODE HERE
  return postings.map(lambda x: (x[0],len(x[1])))

def partition_postings_and_write(postings, offset):
  # YOUR CODE HERE
  rdd = postings.map(lambda x: (token2bucket_id(x[0], offset),(x[0],x[1]))).groupByKey().mapValues(list)
  rdd = rdd.map(lambda x: InvertedIndex.write_a_posting_list(x, bucket_name))
  return rdd

# calc doc_nf [ pineapple ] change in real version
def get_tfidf_norm(tup,w2df,dl):
  w = tup[0]
  pl = tup[1]
  updates = []
  for item in pl: # (doc_id, freq)
    tf = item[1] / dl[item[0]]
    idf = np.log2( len(dl) / w2df[w]) #.get(w,len(d_l_text)))
    updates.append((item[0], (tf * idf)**2))
  return updates
  

In [13]:
offset = 124

In [ ]:
doc_title_pairs = doc_title_pairs.map(lambda x: (x[1], tokenization(x[0])))
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
w2df_title = postings_title.map(lambda x: (x[0],len(x[1])))
w2df_dict_title = w2df_title.collectAsMap()
_ = partition_postings_and_write(postings_title, offset*1).collect()
#title
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

inverted_title = InvertedIndex()
inverted_title.posting_locs = super_posting_locs
inverted_title.df = w2df_dict_title
inverted_title.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

In [ ]:
doc_anchor_pairs = doc_anchor_pairs.map(lambda x: (x[1], tokenization(x[0])))
word_counts_anchor = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)
w2df_anchor = postings_anchor.map(lambda x: (x[0],len(x[1])))
w2df_dict_anchor = w2df_anchor.collectAsMap()
_ = partition_postings_and_write(postings_anchor, offset*2).collect()
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

inverted_anchor = InvertedIndex()
inverted_anchor.posting_locs = super_posting_locs
inverted_anchor.df = w2df_dict_anchor
inverted_anchor.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

In [ ]:
tokenized_docs = doc_text_pairs.map(lambda x: (x[1], tokenization(x[0])))
d_l_text = tokenized_docs.map(lambda x: x[0])
d_l_text = d_l_text.collectAsMap()
word_counts_text = tokenized_docs.flatMap(lambda x: word_count(x[0], x[1])) 
postings_text_unfiltered = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_text = postings_text_unfiltered.filter(lambda x: len(x[1])>50)
w2df_text = postings_text.map(lambda x: (x[0],len(x[1])))
w2df_dict_text = w2df_text.collectAsMap()
doc_nf_dict_text = postings_text.flatMap(lambda x: get_tfidf_norm(x,w2df_dict_text,d_l_text)).reduceByKey(lambda x, y: x+y).collectAsMap()
w2idf_bm25_text = postings_text.map(lambda x: (x[0],np.log(1+((6348910-len(x[1])+0.5)/(len(x[1])+0.5))))).collectAsMap()
_ = partition_postings_and_write(postings_text, offset*0).collect()

In [ ]:
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict_text
# Add dictionary (id_doc,len_doc)
inverted.doc_len = d_l_text
# Add dictionary (id_doc,nf)
inverted.doc_nf = doc_nf_dict_text
# Add dictionary (word,idf_bm25)
inverted.idf_bm25 = w2idf_bm25_text
# write the global stats out
inverted.write_index('.', 'text_index')
# upload to gs
index_src = "text_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
# Put your `generate_graph` function here
def generate_graph(pages):
  # YOUR CODE HERE
  edges = pages.flatMapValues(lambda x:x)
  edges = edges.map(lambda x:(x[0],x[1][0])).distinct()
  first = edges.map(lambda x:(x[0],)).distinct()
  second = edges.map(lambda x:(x[1],)).distinct()
  vertices = sc.union([first,second]).distinct()

  return edges, vertices

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
#pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60

In [ ]:
def write_pkl(dic, base_dir, name):
        """ Write the in-memory index to disk. Results in the file: 
            (1) `name`.pkl containing the global term stats (e.g. df).
        """
        #### GLOBAL DICTIONARIES ####
        _write_globals(dic, base_dir, name)

def _write_globals(dic, base_dir, name):
    with open(Path(base_dir) / f'{name}.pkl', 'wb') as f:
        pickle.dump(dic, f)

In [ ]:
pr_dic = dict(pr.collect())
write_pkl(pr_dic, '.', 'id_pagerank')
# upload to gs
pr_src = "id_pagerank.pkl"
pr_dst = f'gs://{bucket_name}/postings_gcp/{pr_src}'
!gsutil cp $pr_src $pr_dst
!gsutil ls -lh $pr_dst